In [2]:
def getInput():
    return open("../input/day_7.txt").read()

def getInputLines():
    return getInput().split("\n")

# Day 7 Part 1

In [30]:
import re

text = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".strip()

text = getInput()

SYMBOLS = { "2":0, "3": 1, "4":2, "5":3, "6":4, "7":5, "8":6, "9":7,"T":8, "J":9, "Q":10, "K":11, "A":12}
TYPES = { "11111":0, "1112":1, "122":2, "113":3, "23":4, "14":5, "5":6}

def hand_sorting(hand:tuple[int,int,int,int,int]):
    occurrences = [0 for _ in range(13)]
    for card in hand[0]: occurrences[card] += 1
    htype = "".join([str(v) for v in sorted(occurrences) if v > 0])

    return (TYPES[htype], *hand[0])

hands = [(tuple(SYMBOLS[card] for card in cards), int(bid)) for cards, bid in [match.groups() for match in re.finditer(r"([\dTJQKA]+)\s(\d+)", text)]]
winnings = [(i+1) * bid for i,(_, bid) in enumerate(sorted(hands, key=hand_sorting))]

print("result is:", sum(winnings))


result is: 254024898


# Day 6 Part 2

In [61]:
import re

text = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".strip()

text = getInput()

SYMBOLS = { "J":0, "2":1, "3": 2, "4":3, "5":4, "6":5, "7":6, "8":7, "9":8,"T":9, "Q":10, "K":11, "A":12}
TYPES = { "11111":0, "1112":1, "122":2, "113":3, "23":4, "14":5, "5":6}

def realize_joker(occurrences:list[int], hand:tuple[tuple[int], int]):
    cards, bid = hand
    jokers = occurrences[0]
    occurrences[0] = 0
    results = []
    for i in range(1, 13):
        # new_cards = tuple([c + (jokers if c == i else 0) for c in cards])
        new_occ = [cih for cih in occurrences]
        new_occ[i] += jokers
        htype = TYPES["".join([str(v) for v in sorted(new_occ) if v > 0])]
        results.append((htype, *cards))
    return sorted(results)[-1]

def hand_sorting(hand:tuple[tuple[int], int]):
    occurrences = [0 for _ in range(13)]
    has_joker = False
    for card in hand[0]:
        if card < 1: has_joker = True
        occurrences[card] += 1
    if has_joker:
        return realize_joker(occurrences, hand)
    htype = TYPES["".join([str(v) for v in sorted(occurrences) if v > 0])]
    return (htype, *hand[0])

hands = [(tuple(SYMBOLS[card] for card in cards), int(bid)) for cards, bid in [match.groups() for match in re.finditer(r"([\dTJQKA]+)\s(\d+)", text)]]
winnings = [(i+1) * bid for i,(_, bid) in enumerate(sorted(hands, key=hand_sorting))]
print("result is:", sum(winnings))


0.022716103001584997
result is: 254115617


# Ok lemme just dick around a little bit

## Part 1 one liner

In [68]:
sum([(i+1) * bid for i,(_, bid) in enumerate(sorted([(tuple({ "2":0, "3": 1, "4":2, "5":3, "6":4, "7":5, "8":6, "9":7,"T":8, "J":9, "Q":10, "K":11, "A":12}[card] for card in cards), int(bid)) for cards, bid in [match.groups() for match in __import__("re").finditer(r"([\dTJQKA]+)\s(\d+)", open("../input/day_7.txt").read())]], key=lambda x:({ "11111":0, "1112":1, "122":2, "113":3, "23":4, "14":5, "5":6}["".join([str(v) for v in sorted([sum([1 for card in x[0] if card == i]) for i in range(13)]) if v > 0])], *x)))])

254024898

## Part 2 Optimized

@BLANKdev made me want to come back to this

In [62]:
import re

text = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".strip()

text = getInput()

SYMBOLS = { "J":0, "2":1, "3": 2, "4":3, "5":4, "6":5, "7":6, "8":7, "9":8,"T":9, "Q":10, "K":11, "A":12}
TYPES = { "11111":0, "1112":1, "122":2, "113":3, "23":4, "14":5, "5":6}

def hand_sorting(hand:tuple[tuple[int], int]):
    occurrences = [len([1 for card in hand[0] if card == i]) for i in range(13)]
    jokers = occurrences[0]
    occurrences[0] = 0
    max_occ = max(occurrences)
    i = 12
    while i >= 0 and occurrences[i] != max_occ:
        i-=1
    occurrences[i] += jokers
    htype = TYPES["".join([str(v) for v in sorted(occurrences) if v > 0])]
    return (htype, hand[0]), occurrences

hands = [(tuple(SYMBOLS[card] for card in cards), int(bid)) for cards, bid in [match.groups() for match in re.finditer(r"([\dTJQKA]+)\s(\d+)", text)]]
winnings = [(i+1) * bid for i,(_, bid) in enumerate(sorted(hands, key=hand_sorting))]
print("result is:", sum(winnings))
# 254115617

result is: 254115617


## Can I make a part 2 one liner?

In [81]:
import re

def hand_sorting(hand:tuple[tuple[int], int]):
    jokers, *occurrences = [len([1 for card in hand[0] if card == i]) for i in range(13)]
    occurrences[11 - list(reversed(occurrences)).index(max(occurrences))] += jokers
    return ({ "11111":0, "1112":1, "122":2, "113":3, "23":4, "14":5, "5":6}["".join([str(v) for v in sorted(occurrences) if v > 0])], hand[0])

sum([(i+1) * bid for i,(_, bid) in enumerate(sorted([(tuple({ "J":0, "2":1, "3": 2, "4":3, "5":4, "6":5, "7":6, "8":7, "9":8,"T":9, "Q":10, "K":11, "A":12}[card] for card in cards), int(bid)) for cards, bid in [match.groups() for match in re.finditer(r"([\dTJQKA]+)\s(\d+)", open("../input/day_7.txt").read())]], key=hand_sorting))])

254115617

The dumbest way of doing this would be to re-compute occurrences every time it's needed

In [108]:
sum([(i+1) * bid for i,(_, bid) in enumerate(sorted([(tuple({ "J":0, "2":1, "3": 2, "4":3, "5":4, "6":5, "7":6, "8":7, "9":8,"T":9, "Q":10, "K":11, "A":12}[card] for card in cards), int(bid)) for cards, bid in [match.groups() for match in __import__("re").finditer(r"([\dTJQKA]+)\s(\d+)", open("../input/day_7.txt").read())]], key=lambda hand: ({ "11111":0, "1112":1, "122":2, "113":3, "23":4, "14":5, "5":6}["".join([str(v) for e, v in sorted(enumerate([(occ + ([len([1 for card in hand[0] if card == i]) for i in range(13)][0] if idx == 12 - list(reversed([len([1 for card in hand[0] if card == i]) for i in range(13)][1:])).index(max([len([1 for card in hand[0] if card == i]) for i in range(13)][1:])) else 0)) for idx, occ in enumerate([len([1 for card in hand[0] if card == y]) for y in range(13)])][1:]),key=lambda o:o[1]) if v > 0])], hand[0])))])

254115617